# Ring-SIS and Ring-LWE: Making Things Efficient

So we learned about SIS and LWE, but there's a problem - they're kind of slow and need tons of storage. Enter Ring-SIS and Ring-LWE: the structured versions that make everything way more practical.

**Credits to:**  
Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

---

**What we'll cover:**
- Polynomial rings and how they relate to lattices
- Ideal lattices (cyclic and anti-cyclic)
- Ring-SIS and Ring-LWE problems
- Why the "ring" versions are so much faster
- Building actual crypto schemes with them

## Why Regular LWE is Expensive

Let's see the problem first. Regular LWE-based encryption needs a big random matrix $A$.

**Key generation:**
- Pick secret $s \in [-B, B]^n$
- Pick random $A \in \mathbb{Z}_q^{n \times n}$ and error $e \in [-B, B]^n$
- Compute $b = As + e$
- Public key: $(A, b)$, Private key: $s$

The issue? That matrix $A$ takes $n^2$ elements to store! For $n=256$ and $q \approx 2^{16}$, that's 128KB just for one public key.

In [1]:
import numpy as np

# Storage requirements for regular LWE
n_values = [128, 192, 256, 384, 512]
q = 2**16  # typical modulus

print("Regular LWE Storage Requirements:")
print("=" * 60)
print(f"{'n':<10} {'Matrix A size':<20} {'Storage (KB)'}")
print("-" * 60)

for n in n_values:
    matrix_size = n * n
    # 2 bytes per element (for q = 2^16)
    storage_kb = (matrix_size * 2) / 1024
    print(f"{n:<10} {n} × {n:<13} {storage_kb:.1f}")

print("\nThat's a lot of data just for public keys!")
print("Solution: Use structured matrices that can be")
print("represented with just n elements instead of n²")

Regular LWE Storage Requirements:
n          Matrix A size        Storage (KB)
------------------------------------------------------------
128        128 × 128           32.0
192        192 × 192           72.0
256        256 × 256           128.0
384        384 × 384           288.0
512        512 × 512           512.0

That's a lot of data just for public keys!
Solution: Use structured matrices that can be
represented with just n elements instead of n²


## Polynomial Rings: The Key Ingredient

Instead of working with integers in $\mathbb{Z}_q$, we work with polynomials!

A **polynomial ring** $R = \mathbb{Z}[x]/(f)$ consists of polynomials of degree $< n$ where $f$ is a reduction polynomial of degree $n$.

Multiplication works mod $f(x)$:
1. Multiply polynomials normally: $a(x) \times b(x) = h(x)$
2. Divide $h(x)$ by $f(x)$ to get remainder $r(x)$
3. Result is $r(x)$

We can represent polynomials as vectors of coefficients:
$$a(x) = a_0 + a_1x + \cdots + a_{n-1}x^{n-1} \leftrightarrow a = (a_0, a_1, \ldots, a_{n-1})$$

In [2]:
# Polynomial arithmetic example
# R = Z[x]/(x^4 + 2x^2 - 11x + 1)

# Represent polynomials as coefficient vectors
a = np.array([23, 0, 11, 7])      # 23 + 11x^2 + 7x^3
b = np.array([40, 5, 16, 0])      # 40 + 5x + 16x^2

print("Polynomial Ring: R = Z[x]/(x^4 + 2x^2 - 11x + 1)")
print("=" * 60)
print(f"\na(x) = 23 + 11x² + 7x³")
print(f"       coefficients: {a}")
print(f"\nb(x) = 40 + 5x + 16x²")
print(f"       coefficients: {b}")

# Addition and subtraction are easy
a_plus_b = a + b
a_minus_b = a - b

print(f"\nAddition (just add coefficients):")
print(f"a + b = {a_plus_b}")

print(f"\nSubtraction:")
print(f"a - b = {a_minus_b}")

print("\nMultiplication is trickier - need to reduce mod f(x)")
print("Would give: 709 + 2324x + 1618x² + 111x³")
print("(after reduction mod f)")

Polynomial Ring: R = Z[x]/(x^4 + 2x^2 - 11x + 1)

a(x) = 23 + 11x² + 7x³
       coefficients: [23  0 11  7]

b(x) = 40 + 5x + 16x²
       coefficients: [40  5 16  0]

Addition (just add coefficients):
a + b = [63  5 27  7]

Subtraction:
a - b = [-17  -5  -5   7]

Multiplication is trickier - need to reduce mod f(x)
Would give: 709 + 2324x + 1618x² + 111x³
(after reduction mod f)


## Ideal Lattices

An **ideal** $I$ in a polynomial ring $R$ is a special subset where:
- Contains 0
- Closed under addition/subtraction
- If $a \in I$ and $r \in R$, then $a \times r \in I$

The simplest kind is a **principal ideal**: $\langle a(x) \rangle = \{a(x)r(x) \bmod f(x) : r \in R\}$

When we convert an ideal to vectors, we get an **ideal lattice**: $L(I) = \{a : a(x) \in I\}$

These lattices have special structure that we can exploit!

In [3]:
# Example of an ideal
# Let a(x) = 2 + 3x in R = Z[x]/(x^3 - 1)

a = np.array([2, 3, 0])  # 2 + 3x

print("Principal ideal <a(x)> where a(x) = 2 + 3x")
print("R = Z[x]/(x^3 - 1)")
print("=" * 60)

# Generate some elements of the ideal by multiplying by different r(x)
print("\nSome elements a(x) × r(x) in the ideal:")

# r(x) = 1
r1 = np.array([1, 0, 0])
print(f"r(x) = 1:     a×r = {a * r1[0]}")

# r(x) = x
# xa(x) = x(2 + 3x) = 2x + 3x^2 (mod x^3 - 1)
ar_x = np.array([0, 2, 3])
print(f"r(x) = x:     a×r = {ar_x}")

# r(x) = x^2
# x^2 a(x) = 2x^2 + 3x^3 = 2x^2 + 3·1 = 3 + 2x^2 (since x^3 = 1)
ar_x2 = np.array([3, 0, 2])
print(f"r(x) = x²:    a×r = {ar_x2}")

# r(x) = 1 + x
ar_1x = a + ar_x
print(f"r(x) = 1+x:   a×r = {ar_1x}")

print("\nAll these vectors form the ideal lattice L(I)")

Principal ideal <a(x)> where a(x) = 2 + 3x
R = Z[x]/(x^3 - 1)

Some elements a(x) × r(x) in the ideal:
r(x) = 1:     a×r = [2 3 0]
r(x) = x:     a×r = [0 2 3]
r(x) = x²:    a×r = [3 0 2]
r(x) = 1+x:   a×r = [2 5 3]

All these vectors form the ideal lattice L(I)


## Cyclic Lattices: $f(x) = x^n - 1$

When $f(x) = x^n - 1$, we get **cyclic lattices**. If $v$ is in the lattice, so is its right cyclic shift.

Right cyclic shift of $(v_0, v_1, \ldots, v_{n-1})$ is $(v_{n-1}, v_0, v_1, \ldots, v_{n-2})$.

These lattices can be represented by **circulant matrices**. A circulant matrix has the property that each row is a cyclic shift of the previous row.

For polynomial $a(x) = a_0 + a_1x + \cdots + a_{n-1}x^{n-1}$, the circulant matrix is:

$$
A = \text{circ}(a) = \begin{bmatrix}
a_0 & a_{n-1} & a_{n-2} & \cdots & a_1 \\
a_1 & a_0 & a_{n-1} & \cdots & a_2 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
a_{n-1} & a_{n-2} & a_{n-3} & \cdots & a_0
\end{bmatrix}
$$

Key insight: Store just $n$ values instead of $n^2$!

In [4]:
# Building a circulant matrix
def circulant(a):
    """Build circulant matrix from vector a"""
    n = len(a)
    C = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            C[i, j] = a[(i - j) % n]
    return C

# Example: a(x) = 1 + 2x + 3x^2 + 4x^3
a = np.array([1, 2, 3, 4])

print("Circulant matrix for a = [1, 2, 3, 4]")
print("=" * 50)

C = circulant(a)
print("\nC = circ(a):")
print(C)

print("\nNotice the pattern:")
print("- Each row is the previous row shifted right")
print("- Last element wraps to the first position")

# Check multiplication
r = np.array([1, 0, 1, 0])
result = C @ r

print(f"\nMultiply C × r where r = {r}:")
print(f"C × r = {result}")

print("\nThis represents polynomial multiplication mod (x^4 - 1)")
print("a(x) = 1 + 2x + 3x² + 4x³")
print("r(x) = 1 + x²")
print("Result: a(x)×r(x) mod (x^4 - 1)")

Circulant matrix for a = [1, 2, 3, 4]

C = circ(a):
[[1 4 3 2]
 [2 1 4 3]
 [3 2 1 4]
 [4 3 2 1]]

Notice the pattern:
- Each row is the previous row shifted right
- Last element wraps to the first position

Multiply C × r where r = [1 0 1 0]:
C × r = [4 6 4 6]

This represents polynomial multiplication mod (x^4 - 1)
a(x) = 1 + 2x + 3x² + 4x³
r(x) = 1 + x²
Result: a(x)×r(x) mod (x^4 - 1)


## Anti-Cyclic Lattices: $f(x) = x^n + 1$

This is the magic formula used in Ring-SIS and Ring-LWE! With $f(x) = x^n + 1$ (where $n = 2^w$), we get **anti-cyclic lattices**.

If $v = (v_0, v_1, \ldots, v_{n-1})$ is in the lattice, so is $(-v_{n-1}, v_0, v_1, \ldots, v_{n-2})$.

These give us **anti-circulant matrices**:

$$
A = \overline{\text{circ}}(a) = \begin{bmatrix}
a_0 & -a_{n-1} & -a_{n-2} & \cdots & -a_1 \\
a_1 & a_0 & -a_{n-1} & \cdots & -a_2 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
a_{n-1} & a_{n-2} & a_{n-3} & \cdots & a_0
\end{bmatrix}
$$

Notice the minus signs! This prevents the collision attacks that work on cyclic lattices.

In [5]:
# Building an anti-circulant matrix
def anti_circulant(a):
    """Build anti-circulant matrix from vector a"""
    n = len(a)
    C = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if i >= j:
                C[i, j] = a[i - j]
            else:
                C[i, j] = -a[n + i - j]
    return C

# Example: a(x) = 3 - 2x + 5x^2 - 4x^3
a = np.array([3, -2, 5, -4])
n = 4

print("Anti-circulant matrix for a = [3, -2, 5, -4]")
print("Polynomial: a(x) = 3 - 2x + 5x² - 4x³")
print("Ring: R = Z[x]/(x^4 + 1)")
print("=" * 60)

A = anti_circulant(a)
print("\nA = anti-circ(a):")
print(A)

print("\nKey differences from circulant:")
print("- Wrapped elements get negated")
print("- Row sums are NOT all equal")
print("- This prevents collision attacks!")

# Test multiplication
r = np.array([1, 2, 11, -7])
result = A @ r

print(f"\nMultiply A × r where r = {r}:")
print(f"A × r = {result}")

print("\nThis computes a(x) × r(x) mod (x^4 + 1)")
print("where r(x) = 1 + 2x + 11x² - 7x³")

Anti-circulant matrix for a = [3, -2, 5, -4]
Polynomial: a(x) = 3 - 2x + 5x² - 4x³
Ring: R = Z[x]/(x^4 + 1)

A = anti-circ(a):
[[ 3  4 -5  2]
 [-2  3  4 -5]
 [ 5 -2  3  4]
 [-4  5 -2  3]]

Key differences from circulant:
- Wrapped elements get negated
- Row sums are NOT all equal
- This prevents collision attacks!

Multiply A × r where r = [ 1  2 11 -7]:
A × r = [-58  83   6 -37]

This computes a(x) × r(x) mod (x^4 + 1)
where r(x) = 1 + 2x + 11x² - 7x³


## Ring-SIS: The Structured Version

Now we're ready for Ring-SIS! Instead of a random matrix $A$, we use concatenated anti-circulant matrices.

**$Ring-SIS(n, \ell, q, B)$:** Given polynomials $a_1, \ldots, a_\ell \in R_q = \mathbb{Z}_q[x]/(x^n + 1)$, find polynomials $z_1, \ldots, z_\ell \in R_q$ (not all zero) such that:

$$
a_1z_1 + a_2z_2 + \cdots + a_\ell z_\ell = 0 \pmod{q}
$$

with $\|z_i\|_\infty \leq B$ (all coefficients small).

In matrix form: Find $z \in [-B, B]^m$ (where $m = \ell n$) such that $Az = 0 \pmod{q}$, where:

$$
A = [\overline{\text{circ}}(a_1) | \overline{\text{circ}}(a_2) | \cdots | \overline{\text{circ}}(a_\ell)]
$$

This is SIS but with structured matrices!

In [6]:
# Ring-SIS example (small parameters)
q = 59
n = 4
ell = 3
B = 2

# Three polynomials defining the Ring-SIS instance
a1 = np.array([10, 0, 16, 51])     # 10 + 16x² + 51x³
a2 = np.array([41, 10, 54, 16])    # 41 + 10x + 54x² + 16x³
a3 = np.array([11, 17, 39, 5])     # 11 + 17x + 39x² + 5x³

print("Ring-SIS Instance")
print("=" * 60)
print(f"Parameters: q={q}, n={n}, ℓ={ell}, B={B}")
print(f"Ring: R_q = Z_{q}[x]/(x^{n} + 1)")

print(f"\nGiven polynomials:")
print(f"a₁(x) = {a1[0]} + {a1[2]}x² + {a1[3]}x³")
print(f"a₂(x) = {a2[0]} + {a2[1]}x + {a2[2]}x² + {a2[3]}x³")
print(f"a₃(x) = {a3[0]} + {a3[1]}x + {a3[2]}x² + {a3[3]}x³")

# Build the structured matrix A
A1 = anti_circulant(a1)
A2 = anti_circulant(a2)
A3 = anti_circulant(a3)
A = np.hstack([A1, A2, A3]) % q

print(f"\nMatrix A is {n}×{ell*n} = {n}×{ell*n}")
print("A = [anti-circ(a₁) | anti-circ(a₂) | anti-circ(a₃)]")

print("\nGoal: Find z ∈ [-2,2]^12 with Az = 0 (mod 59)")
print("      and not all z_i are zero")

# One known solution (from the example)
z1 = np.array([1, 2, -1, 2])
z2 = np.array([-1, 2, 0, -2])
z3 = np.array([1, 0, 0, 0])
z_solution = np.concatenate([z1, z2, z3])

result = (A @ z_solution) % q

print(f"\nFound solution:")
print(f"z₁(x) = 1 + 2x - x² + 2x³")
print(f"z₂(x) = -1 + 2x - 2x³")
print(f"z₃(x) = 1")
print(f"\nVerification: Az mod {q} = {result}")
print("All zeros!" if np.all(result == 0) else "Doesn't work")

Ring-SIS Instance
Parameters: q=59, n=4, ℓ=3, B=2
Ring: R_q = Z_59[x]/(x^4 + 1)

Given polynomials:
a₁(x) = 10 + 16x² + 51x³
a₂(x) = 41 + 10x + 54x² + 16x³
a₃(x) = 11 + 17x + 39x² + 5x³

Matrix A is 4×12 = 4×12
A = [anti-circ(a₁) | anti-circ(a₂) | anti-circ(a₃)]

Goal: Find z ∈ [-2,2]^12 with Az = 0 (mod 59)
      and not all z_i are zero

Found solution:
z₁(x) = 1 + 2x - x² + 2x³
z₂(x) = -1 + 2x - 2x³
z₃(x) = 1

Verification: Az mod 59 = [0 0 0 0]
All zeros!


## Ring-SIS for Hash Functions

Ring-SIS gives us collision-resistant hash functions with provable security!

**Setup:** Pick polynomials $a_1, \ldots, a_\ell \in R_q$ and build $A = [\overline{\text{circ}}(a_1) | \cdots | \overline{\text{circ}}(a_\ell)]$

**Hash function:** $H_A(z) = Az \bmod q$ 

This compresses $m$ bits to $n \log_2 q$ bits (where $m = \ell n$).

**Security:** Finding a collision means solving Ring-SIS, which is as hard as worst-case SVP in anti-cyclic lattices.

Key advantage over regular SIS: The matrix $A$ can be stored with just $\ell n$ elements instead of $mn = \ell n^2$ elements!

In [7]:
# Comparing storage: SIS vs Ring-SIS
n_values = [128, 256, 512]
ell = 4
q_bits = 16  # q ≈ 2^16

print("Storage Comparison: SIS vs Ring-SIS")
print("=" * 70)
print(f"Parameters: ℓ={ell}, q ≈ 2^{q_bits}")
print()
print(f"{'n':<10} {'SIS Matrix':<20} {'Ring-SIS':<20} {'Reduction'}")
print("-" * 70)

for n in n_values:
    m = ell * n
    
    # SIS: store full n×m matrix
    sis_elements = n * m
    sis_kb = (sis_elements * 2) / 1024  # 2 bytes per element
    
    # Ring-SIS: store ℓ polynomials of degree n
    ringsis_elements = ell * n
    ringsis_kb = (ringsis_elements * 2) / 1024
    
    reduction = sis_kb / ringsis_kb
    
    print(f"{n:<10} {sis_kb:>6.1f} KB{'':<12} {ringsis_kb:>6.1f} KB{'':<12} {reduction:.0f}×")

print("\nRing-SIS stores n times less data!")
print("For n=256, that's 256KB vs 1KB")

Storage Comparison: SIS vs Ring-SIS
Parameters: ℓ=4, q ≈ 2^16

n          SIS Matrix           Ring-SIS             Reduction
----------------------------------------------------------------------
128         128.0 KB                1.0 KB             128×
256         512.0 KB                2.0 KB             256×
512        2048.0 KB                4.0 KB             512×

Ring-SIS stores n times less data!
For n=256, that's 256KB vs 1KB


## Ring-LWE: Efficient Encryption

Ring-LWE is the structured version of LWE. Instead of matrix-vector products, we do polynomial multiplication!

**Ring-LWE$(n, k, q, B)$:** Given polynomials $(a_1, b_1), \ldots, (a_k, b_k)$ where $b_i = a_i s + e_i$ in $R_q$, find the secret polynomial $s$.

Here $s, e_i \in S_B$ (polynomials with small coefficients in $[-B, B]$).

In matrix form, this is solving:

$$
As + e = b \pmod{q}
$$

where $A$ is a $kn \times n$ matrix built from anti-circulant blocks.

Just like Ring-SIS, this is LWE with structured matrices!

In [8]:
# Ring-LWE example (small parameters)
q = 17
n = 4
k = 3
B = 3

print("Ring-LWE Instance")
print("=" * 60)
print(f"Parameters: q={q}, n={n}, k={k}, B={B}")
print(f"Ring: R_q = Z_{q}[x]/(x^{n} + 1)")

# Given polynomials (simplified example)
a1 = np.array([3, 7, 12, 5])
a2 = np.array([9, 2, 15, 11])
a3 = np.array([6, 14, 8, 4])

b1 = np.array([13, 12, 7, 16])
b2 = np.array([8, 14, 10, 1])
b3 = np.array([15, 3, 11, 9])

print("\nGiven: (aᵢ, bᵢ) pairs where bᵢ = aᵢs + eᵢ")
print(f"a₁ = {a1}, b₁ = {b1}")
print(f"a₂ = {a2}, b₂ = {b2}")
print(f"a₃ = {a3}, b₃ = {b3}")

# Solution (from example)
s = np.array([5, 15, 15, 12])
e1 = np.array([2, 0, -3, 1])
e2 = np.array([-2, 1, 2, -2])
e3 = np.array([-2, 1, 1, -3])

print(f"\nSecret polynomial: s = {s}")
print(f"Error polynomials:")
print(f"  e₁ = {e1}")
print(f"  e₂ = {e2}")
print(f"  e₃ = {e3}")

# Verify: compute a_i * s + e_i and check if it equals b_i
def poly_mult_mod(a, s, q, n):
    """Polynomial multiplication mod (x^n + 1) and mod q"""
    A = anti_circulant(a)
    return (A @ s) % q

check1 = (poly_mult_mod(a1, s, q, n) + e1) % q
check2 = (poly_mult_mod(a2, s, q, n) + e2) % q
check3 = (poly_mult_mod(a3, s, q, n) + e3) % q

print(f"\nVerification:")
print(f"a₁s + e₁ mod {q} = {check1} (should be {b1})")
print(f"a₂s + e₂ mod {q} = {check2} (should be {b2})")
print(f"a₃s + e₃ mod {q} = {check3} (should be {b3})")

if np.all(check1 == b1) and np.all(check2 == b2) and np.all(check3 == b3):
    print("\nAll checks pass!")

Ring-LWE Instance
Parameters: q=17, n=4, k=3, B=3
Ring: R_q = Z_17[x]/(x^4 + 1)

Given: (aᵢ, bᵢ) pairs where bᵢ = aᵢs + eᵢ
a₁ = [ 3  7 12  5], b₁ = [13 12  7 16]
a₂ = [ 9  2 15 11], b₂ = [ 8 14 10  1]
a₃ = [ 6 14  8  4], b₃ = [15  3 11  9]

Secret polynomial: s = [ 5 15 15 12]
Error polynomials:
  e₁ = [ 2  0 -3  1]
  e₂ = [-2  1  2 -2]
  e₃ = [-2  1  1 -3]

Verification:
a₁s + e₁ mod 17 = [ 1 14 11  7] (should be [13 12  7 16])
a₂s + e₂ mod 17 = [3 5 8 8] (should be [ 8 14 10  1])
a₃s + e₃ mod 17 = [ 3  5  4 11] (should be [15  3 11  9])


## Ring-LWE Public-Key Encryption

Now for the practical crypto! Here's how to build encryption with Ring-LWE.

**Key Generation:**
1. Pick secret $s \in S_B$ (polynomial with small coefficients)
2. Pick random $a \in R_q$ and error $e \in S_B$
3. Compute $b = as + e \in R_q$
4. Public key: $(a, b)$, Private key: $s$

**Encryption** (message $m \in \{0,1\}^n$):
1. Get Alice's public key $(a, b)$
2. Pick random $r, z, z' \in S_B$
3. Compute $c_1 = ar + z$ and $c_2 = br + z' + \lfloor q/2 \rfloor m$
4. Ciphertext: $(c_1, c_2)$

**Decryption:**
1. Compute $c_2 - sc_1$
2. Round each coefficient: if close to 0, output 0; if close to $q/2$, output 1

The magic: $c_2 - sc_1 = (br + z') - s(ar + z) + \lfloor q/2 \rfloor m = e \cdot r + z' - sz + \lfloor q/2 \rfloor m$, which is approximately $\lfloor q/2 \rfloor m$ if errors are small!

In [9]:
# Ring-LWE encryption example
np.random.seed(42)

q = 97
n = 8
B = 3

print("Ring-LWE Encryption Scheme")
print("=" * 60)
print(f"Parameters: q={q}, n={n}, B={B}")
print(f"Ring: R_q = Z_{q}[x]/(x^{n} + 1)")

# Key Generation
s = np.random.randint(-B, B+1, n)  # Secret
a = np.random.randint(0, q, n)     # Random polynomial
e = np.random.randint(-B, B+1, n)  # Error

b = (poly_mult_mod(a, s, q, n) + e) % q

print("\n--- Key Generation ---")
print(f"Secret s: {s}")
print(f"Public a: {a}")
print(f"Error e: {e}")
print(f"Public b = as + e: {b}")

# Encryption
m = np.array([1, 0, 1, 1, 0, 0, 1, 0])  # Message bits
print(f"\n--- Encryption ---")
print(f"Message: {m}")

r = np.random.randint(-B, B+1, n)
z = np.random.randint(-B, B+1, n)
z_prime = np.random.randint(-B, B+1, n)

c1 = (poly_mult_mod(a, r, q, n) + z) % q
c2 = (poly_mult_mod(b, r, q, n) + z_prime + (q // 2) * m) % q

print(f"Random r: {r}")
print(f"Ciphertext c1: {c1}")
print(f"Ciphertext c2: {c2}")

# Decryption
print("\n--- Decryption ---")
sc1 = poly_mult_mod(c1, s, q, n)
decrypted_raw = (c2 - sc1) % q

# Adjust to centered representation for rounding
decrypted_centered = np.where(decrypted_raw > q//2, decrypted_raw - q, decrypted_raw)

# Round to nearest multiple of q/2
m_recovered = np.where(np.abs(decrypted_centered) < q//4, 0, 1)

print(f"c2 - s·c1 (mod q): {decrypted_raw}")
print(f"Recovered message: {m_recovered}")
print(f"Original message:  {m}")

if np.all(m == m_recovered):
    print("\nDecryption successful!")
else:
    print("\nDecryption failed (errors too large)")

Ring-LWE Encryption Scheme
Parameters: q=97, n=8, B=3
Ring: R_q = Z_97[x]/(x^8 + 1)

--- Key Generation ---
Secret s: [ 3  0  1  3 -1  1  1  3]
Public a: [82 86 74 74 87 23  2 21]
Error e: [ 1 -2  0  2  2 -2  0  1]
Public b = as + e: [51 63  8  0 33 41 55  1]

--- Encryption ---
Message: [1 0 1 1 0 0 1 0]
Random r: [-3  0 -2  2  1  0 -3 -3]
Ciphertext c1: [ 7  6 56 77 17 86 56 25]
Ciphertext c2: [ 8 77 63 50 59 30 63 24]

--- Decryption ---
c2 - s·c1 (mod q): [47 40 47 57 86 10 56  0]
Recovered message: [1 1 1 1 0 0 1 0]
Original message:  [1 0 1 1 0 0 1 0]

Decryption failed (errors too large)


## Why Ring Versions Win

Let's summarize why Ring-SIS and Ring-LWE are so much better than regular SIS/LWE:

**Storage:**
- SIS/LWE: Store $n \times m$ matrix = $nm$ elements
- Ring-SIS/Ring-LWE: Store $\ell$ polynomials = $\ell n$ elements
- Savings: Factor of $n$ reduction!

**Computation:**
- SIS/LWE: Matrix-vector multiplication = $O(nm)$ operations
- Ring-SIS/Ring-LWE: Polynomial multiplication = $O(n \log n)$ with FFT/NTT
- Savings: Much faster for large $n$

**Security:**
- Both have worst-case to average-case reductions
- Ring versions: Based on SVP in structured (anti-cyclic) lattices
- Regular versions: Based on SVP in general lattices
- No known attacks exploit the structure!

The structure makes everything practical without (apparently) weakening security.

In [10]:
# Comprehensive comparison
n_vals = [128, 256, 512, 1024]
ell = 4

print("Comprehensive Comparison: Regular vs Ring Versions")
print("=" * 80)
print(f"Parameters: ℓ={ell}, q ≈ 2^16\n")

print(f"{'n':<8} {'Storage (KB)':<25} {'Mult Ops':<30} {'Savings'}")
print(f"{'':8} {'Regular':<12} {'Ring':<13} {'Regular':<15} {'Ring':<15}")
print("-" * 80)

for n in n_vals:
    m = ell * n
    
    # Storage
    regular_storage = (n * m * 2) / 1024
    ring_storage = (ell * n * 2) / 1024
    storage_factor = regular_storage / ring_storage
    
    # Operations
    regular_ops = n * m
    ring_ops = n * np.log2(n) * ell  # Using NTT
    ops_factor = regular_ops / ring_ops
    
    print(f"{n:<8} {regular_storage:>8.1f} KB   {ring_storage:>8.1f} KB   "
          f"{regular_ops:>10.0f}      {ring_ops:>10.0f}      "
          f"{storage_factor:.0f}× / {ops_factor:.0f}×")

print("\nFor n=1024:")
print("  Regular: 8192 KB storage, ~4 million operations")
print("  Ring: 8 KB storage, ~40k operations")
print("  That's 1000× less storage and 100× faster!")

print("\nThis is why Ring-LWE is used in real post-quantum schemes")

Comprehensive Comparison: Regular vs Ring Versions
Parameters: ℓ=4, q ≈ 2^16

n        Storage (KB)              Mult Ops                       Savings
         Regular      Ring          Regular         Ring           
--------------------------------------------------------------------------------
128         128.0 KB        1.0 KB        65536            3584      128× / 18×
256         512.0 KB        2.0 KB       262144            8192      256× / 32×
512        2048.0 KB        4.0 KB      1048576           18432      512× / 57×
1024       8192.0 KB        8.0 KB      4194304           40960      1024× / 102×

For n=1024:
  Regular: 8192 KB storage, ~4 million operations
  Ring: 8 KB storage, ~40k operations
  That's 1000× less storage and 100× faster!

This is why Ring-LWE is used in real post-quantum schemes


## Wrapping Up

So what did we learn about Ring-SIS and Ring-LWE?

**The core idea:** Replace random matrices with structured matrices built from polynomials. This gives us:
- Massive storage savings (factor of $n$)
- Much faster operations (FFT/NTT magic)
- Same security guarantees (worst-case lattice hardness)

**Ring-SIS:** Used for collision-resistant hashing and signatures. Solving it means breaking worst-case SVP in anti-cyclic lattices.

**Ring-LWE:** Used for public-key encryption. Breaking it means breaking worst-case SIVP in anti-cyclic lattices (quantum reduction).

**The polynomial ring trick:** Work in $R_q = \mathbb{Z}_q[x]/(x^n + 1)$ where $n = 2^w$. This gives anti-circulant matrices that prevent collision attacks while maintaining efficiency.

**Real-world impact:** Ring-LWE variants are the basis for NIST's post-quantum encryption standards (Kyber/ML-KEM). The structured approach made lattice crypto practical enough to deploy worldwide.

The structure doesn't seem to make these problems easier - no attacks exploit it. We get efficiency basically for free!